In [32]:
%run data.ipynb  # from data import pets_df, foods_df

In [33]:
class Animal():
    def __init__(self, species, lvl=1, atk=None, hp=None):
        assert species in pets_df.index.get_level_values('Name'), f'{species} is not a valid species'
        assert atk is None or isinstance(atk, (int, np.integer)), f'{atk} is not a valid atk'
        assert hp is None or isinstance(hp, (int, np.integer)), f'{hp} is not a valid hp'
        
        species_data = pets_df.loc[species]
        data = species_data.loc[lvl]
        
        self.species = species
        self.atk = data.Attack if atk is None else atk
        self.hp = data.Health if hp is None else hp
        self.tier = data.Tier
        
    @property
    def atk(self):
        return self._atk
    @atk.setter
    def atk(self, new_atk):
        self._atk = min(50, new_atk)
        
    @property
    def hp(self):
        return self._hp
    @hp.setter
    def hp(self, new_hp):
        self._hp = min(50, new_hp)
        
    def __repr__(self):
        return f'{self.species} ({self.atk},{self.hp})'
    
    def buff_stats(self, atk=0, hp=0, temp=False):
        assert isinstance(self, Pet) or not temp, 'Cannot add temp stats to shop animals!'
        if temp:
            self.temp_atk += atk
            self.temp_hp += hp
        else:
            self.atk += atk
            self.hp += hp

In [52]:
class Pet(Animal):
    def __init__(self, species, lvl=1, atk=None, hp=None, status=None, exp=0):
        assert lvl in [1, 2, 3], f'{lvl} is not a valid level'
        assert isinstance(exp, (int, np.integer)), f'{exp} is not a valid exp'
        
        super().__init__(species, lvl, atk, hp)
        species_data = pets_df.loc[species]
        self.pos = None
        self.temp_atk = 0
        self.temp_hp = 0
        self.trigger_dict = {'StartOfBattle': 0, 'BeforeClash': 0, 'AfterClash': 0,
                             'Hurt': 0, 'Faint': 0, 'KnockOut': 0, 
                             'Transformed': 0, 'Pushed': 0, 'Summoned': 0, 'DealtDamage': 0,
                             'Buy': 0, 'Sell': 0, 'LevelUp': 0, 'EatsShopFood': 0, 'EatsApple': 0}
        
        self.perm_abilities = {i: make_abl_list(self, *species_data.loc[i, 'Trigger':'N_charges']) for i in [1, 2, 3]}
        self.temp_abilities = None
        self.lvl = lvl
        self.exp = exp
        self.status = status
        
    @property
    def abilities(self):
        if self.temp_abilities is None:
            return self.perm_abilities
        else: 
            return self.temp_abilities
        
    @property
    def ability(self):
        if self.status_ability is None:
            return self.abilities[self.lvl]
        else:
            return self.abilities[self.lvl] + self.status_ability
                
    @property
    def exp(self):
        return self._exp
    @exp.setter
    def exp(self, new_exp):
        self._exp = new_exp
        if self.lvl == 3:
            self._exp = 0
            return
        if self.lvl == 1 and self._exp >= 2:
            self.lvl = 2
            self._exp = new_exp - 2
            self.trigger_dict['LevelUp'] = 1
        if self.lvl == 2 and self._exp >= 3:
            self.lvl = 3
            self._exp = 0
            # TODO: see if jellyfish buffs a double level-up twice
            self.trigger_dict['LevelUp'] = 1  # or +=
            
    @property
    def status(self):
        return self._status
    @status.setter
    def status(self, new_status):
        if isinstance(new_status, Status):
            self._status = new_status
        elif isinstance(new_status, str):
            self._status = Status(new_status)
        elif new_status is None:
            self._status = Status('NoStatus')
        else:
            assert False, f'{self.species} was given a bad status'
            
        if self._status.is_ability:
            self.status_ability = [Ability(self, **self._status.is_ability)]
        else:
            self.status_ability = None
                
    def __str__(self):
        atk_str = f'{self.atk}+{self.temp_atk}' if self.temp_atk > 0 else str(self.atk)
        hp_str = f'{self.hp}+{self.temp_hp}' if self.temp_hp > 0 else str(self.hp)
        
        if self.status.name == 'NoStatus':
            return f'Lv.{self.lvl} {self.species} ({atk_str},{hp_str})'
        else:
            return f'Lv.{self.lvl} {self.species} ({atk_str},{hp_str}) with {self.status.name}'

    def __repr__(self):
        return f'{self.species}: lvl={self.lvl}, atk={self.atk}+{self.temp_atk}, hp={self.hp}+{self.temp_hp}, status={self.status}, exp={self.exp}'

    def take_damage(self, n, peanut=False):  
        assert n > 0, f'Pet cannot take {n} damage!'
        orig_hp = self.hp
        
        if self.status.prevents_damage and self.status.n_charges > 0:
            self.status.n_charges -= 1
            if self.status.prevents_damage['full_shield']:
                self.hp -= max(0, n - self.status.prevents_damage['n'])
            else:
                self.hp -= max(1, n - self.status.prevents_damage['n'])
        elif self.status.name == 'Pepper':
            self.status.n_charges -= 1
            self.hp = max(1, self.hp - n)
        else:
            self.hp -= n

        if peanut and self.hp < orig_hp:
            self.hp = 0
            
        if self.hp <= 0:
            self.trigger_dict['Faint'] = 1
        elif self.hp < orig_hp:
            self.trigger_dict['Hurt'] += 1
            
        return self.hp
            
    def deal_damage(self):
        if self.status.does_damage and self.status.n_charges > 0:
            self.status.n_charges -= 1
            info = self.status.does_damage
            if info.get('cond_func') and not info['cond_func']():
                return self.atk
            else:
                if info.get('n'):
                    return self.atk + info['n']
                if info.get('mult'):
                    return self.atk * info['mult']
            assert False, f'{self} has status {self.status} but faulty info dictionary!'
        else:
            return self.atk


In [56]:
w = Pet('Wombat', status='Honey')
w.temp_abilities = {lvl: make_abl_list(w, *pets_df.loc[('Piranha', lvl), 'Trigger':'N_charges'])
                                       for lvl in [1, 2, 3]}
w.ability

Used temp_abilies! :)


[Hurt Self: Wombat do ModifyStats on AllFriends,
 Faint Self: Wombat do ModifyStats on AllFriends,
 Faint Self: Wombat do Summon on Self]

In [6]:
class ShopAnimal(Animal):
    def __init__(self, species, atk=None, hp=None):
        super().__init__(species, 1, atk, hp)
        
        self.cost = 3
        self.is_frozen = False
        
    def __repr__(self):
        return super().__repr__() + (' (frozen)' if self.is_frozen else '')
        
    def convert_to_pet(self):
        return Pet(self.species, atk=self.atk, hp=self.hp)